<a href="https://colab.research.google.com/github/nafisenik/DataMinig_pySpark/blob/main/question_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install pySpark

In [1]:
! pip install -q pyspark

In [2]:
! apt-get install openjdk-8-jdk-headless -qq > /dev/null
! wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
! tar xf spark-3.2.1-bin-hadoop3.2.tgz

In [3]:
!pip install hazm

     |████████████████████████████████| 316 kB 5.0 MB/s 
     |████████████████████████████████| 1.4 MB 53.3 MB/s 
     |████████████████████████████████| 233 kB 51.6 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394487 sha256=4f648a7911c405e5b1b5ae2f843832741c4b9635b14a90a0f19ceae373feb79b
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154070 sha256=699b165f75a0784b56f0590668668a83af270a1f83b4a9122f6624073ef5a9ba
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


### Import Libraries

In [4]:
import random
import statistics
import time
import numpy as np
from operator import add
import matplotlib.pyplot as plt
import math
from pyspark.sql import SparkSession
import pandas as pd
import pyspark.sql.functions as F
import re
from hazm import *
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf
from pyspark.sql.functions import col
from hazm import sent_tokenize, word_tokenize
from hazm import stopwords_list
from hazm import Stemmer, Lemmatizer

In [5]:
# First Method
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .getOrCreate()

sc = spark.sparkContext

In [6]:
! wget -q https://raw.githubusercontent.com/SajjadMb/PySpark-walkthrough/main/data/digikala_comments.csv

In [7]:
df = spark.read.option("header", True).option("multiline",True).csv("digikala_comments.csv")
df.show()


+----------+--------------------+--------+-------+-----+--------+-------------------+---------------+--------------------+--------------------+--------------------+--------------------+
|product_id|       product_title|title_en|user_id|likes|dislikes|verification_status|      recommend|               title|             comment|          advantages|       disadvantages|
+----------+--------------------+--------+-------+-----+--------+-------------------+---------------+--------------------+--------------------+--------------------+--------------------+
|      3692|ماوس بی‌سیم لاجیت...|      IT| 989472|    0|       0|           verified|             \N|                null|واقعا عالیه. من ک...|                null|                null|
|     90213|شارژر همراه شیاوم...|      AC|3862150|    4|       1|           verified|    recommended|        واقعاً عالیه|سلام، قبل اینکه ن...|  "[""عمر طولانی\r""|""افت بسیار کم می...|
|     59473|یدک پولیشر میکروف...|      HW| 626843|    1|       0|     

In [8]:
def convert_num(text):
  persian = ['۰', '۱', '۲', '۳', '۴', '۵', '۶', '۷', '۸', '۹']
  convertedPersianNums = text.replace('۰','0').replace('۱','1').replace('۲','2').replace('۳','3').replace('۴','4')\
  .replace('۵','5').replace('۶','6').replace('۷','7').replace('۸','8').replace('۹','9')
  return convertedPersianNums


In [9]:
stopwords = stopwords_list()+['ها','های','یه','ک','ام','ای','اگه','تر','اس','اینو','ازش','واسه','ی','هاش','برا','تره','اونم','چی','ترین','جی','داره','هست']
def remove_stop_words(text):
  text = re.sub('[^آ-ی]', ' ', text)
  tokenized_text = word_tokenize(text)
  #print(tokenized_text)
  all_tokens = [t for t in tokenized_text if t not in stopwords]
  #print(all_tokens)
  text = " ".join(all_tokens)
  return text

In [10]:
x = remove_stop_words('من از شهر به روستا رفتم و در راه علی را دیدم')
print(stopwords)

['و', 'در', 'به', 'از', 'که', 'این', 'را', 'با', 'است', 'برای', 'آن', 'یک', 'خود', 'تا', 'کرد', 'بر', 'هم', 'نیز', 'گفت', 'می\u200cشود', 'وی', 'شد', 'دارد', 'ما', 'اما', 'یا', 'شده', 'باید', 'هر', 'آنها', 'بود', 'او', 'دیگر', 'دو', 'مورد', 'می\u200cکند', 'شود', 'کند', 'وجود', 'بین', 'پیش', 'شده_است', 'پس', 'نظر', 'اگر', 'همه', 'یکی', 'حال', 'هستند', 'من', 'کنند', 'نیست', 'باشد', 'چه', 'بی', 'می', 'بخش', 'می\u200cکنند', 'همین', 'افزود', 'هایی', 'دارند', 'راه', 'همچنین', 'روی', 'داد', 'بیشتر', 'بسیار', 'سه', 'داشت', 'چند', 'سوی', 'تنها', 'هیچ', 'میان', 'اینکه', 'شدن', 'بعد', 'جدید', 'ولی', 'حتی', 'کردن', 'برخی', 'کردند', 'می\u200cدهد', 'اول', 'نه', 'کرده_است', 'نسبت', 'بیش', 'شما', 'چنین', 'طور', 'افراد', 'تمام', 'درباره', 'بار', 'بسیاری', 'می\u200cتواند', 'کرده', 'چون', 'ندارد', 'دوم', 'بزرگ', 'طی', 'حدود', 'همان', 'بدون', 'البته', 'آنان', 'می\u200cگوید', 'دیگری', 'خواهد_شد', 'کنیم', 'قابل', 'یعنی', 'رشد', 'می\u200cتوان', 'وارد', 'کل', 'ویژه', 'قبل', 'براساس', 'نیاز', 'گذاری', 'هنوز', '

In [11]:
#https://github.com/optimopium/PySpark-walkthrough/blob/main/Spark_Tutorial.ipynb
def clean_normalized(text):
  normalizer = Normalizer()
  try:
    #normalize
    cleaned_text = text.replace('_x000D_\n',' ').splitlines() 
    cleaned_text = ' '.join(cleaned_text)
    cleaned_text = normalizer.normalize(cleaned_text)
    cleaned_text = cleaned_text.replace("،"," ").replace("\u200c"," ").replace(r"\u200"," ").replace(r"\u200b"," ")\
            .replace("."," ").replace("-"," ").replace("_"," ").replace("\n"," ").replace("\r"," ").replace("\\"," ")\
            .replace("("," ").replace(")"," ").replace("["," ").replace("!"," ").replace("]"," ")\
            .replace("r"," ").replace("\""," ")
    cleaned_text = re.sub(r"\s+.\u0648.\s+", ' ', cleaned_text)
    cleaned_text = re.sub(r'[^\w\s]', ' ', cleaned_text)

    #Remove numbers
    #cleaned_text = cleaned_text.replace('x000D',' ').splitlines() 
    #cleaned_text = ' '.join(cleaned_text)
    cleaned_text = convert_num(cleaned_text)
    cleaned_text = re.sub(r'[0-9]', ' ', cleaned_text)
    cleaned_text = re.sub(' +', ' ',cleaned_text)
    cleaned_text = remove_stop_words(cleaned_text)
    return cleaned_text
  except:
    return None

In [12]:
clean_comment = udf(lambda z: clean_normalized(z))
stop_words_deletion = udf(lambda z: remove_stop_words(z))

In [13]:
df.select('product_id', 'comment').show(truncate=False)
df_res = df.select(col("product_id"),clean_comment(col("comment")).alias("comment") ) 


+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [14]:
df_res.show(truncate=False)

+----------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|product_id|comment                                                                                                                                                                                                  

In [15]:
selected_df = df.select("product_id", "likes", "dislikes", "comment")
popularity = selected_df.dropna().withColumn("popularity",selected_df["likes"] - selected_df["dislikes"])
popularity.orderBy("popularity", ascending=False).select('product_id','popularity').show(10)


+----------+----------+
|product_id|popularity|
+----------+----------+
|    517207|     184.0|
|    597475|     124.0|
|    503608|     115.0|
|    701679|     114.0|
|    133901|     113.0|
|    517207|     109.0|
|    443966|     108.0|
|    226249|     102.0|
|    341816|      91.0|
|    313420|      80.0|
+----------+----------+
only showing top 10 rows



In [16]:
order_product = popularity.orderBy("popularity", ascending=False)
my_product = order_product.take(50)
products = spark.createDataFrame(my_product)
products.show()

+----------+-----+--------+--------------------+----------+
|product_id|likes|dislikes|             comment|popularity|
+----------+-----+--------+--------------------+----------+
|    517207|  206|      22|خلاصه بگم ماشین ب...|     184.0|
|    597475|  129|       5|با سلام._x000D_\n...|     124.0|
|    503608|  125|      10|    هیچ تاثیری نداشت|     115.0|
|    701679|  145|      31|من این مداد رنگی ...|     114.0|
|    133901|  122|       9|این دوربین قطعا ا...|     113.0|
|    517207|  136|      27|خیلی وسیله ی بی ک...|     109.0|
|    443966|  110|       2|اینا به این درد م...|     108.0|
|    226249|  105|       3|سلام دوستان_x000D...|     102.0|
|    341816|   96|       5|ببینید اینکه یکی ...|      91.0|
|    313420|  100|      20|    بهقیمتش نمی عرضه|      80.0|
|    253710|   84|       8|دوستان دیر شارژ ش...|      76.0|
|    203451|   67|       3|من تقریبا یک سال ...|      64.0|
|    334514|   72|       9|این برندحرف نداره...|      63.0|
|    700304|   60|       2|در یک کلام ، 

In [17]:
filter_products = products.select('product_id','comment')
filter_products.show(truncate=False)

+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [18]:
df_word_count = df_res.withColumn('wordCount', F.size(F.split(F.col('comment'), ' ')))
df_word_count.select('product_id','wordCount').show(100)

+----------+---------+
|product_id|wordCount|
+----------+---------+
|      3692|        3|
|     90213|       93|
|     59473|       30|
|    120499|      129|
|     67200|       38|
|    133722|      127|
|    148509|       20|
|     56871|        9|
|     49738|       22|
|    161548|        1|
|      5107|       41|
|    156041|       10|
|    134367|       44|
|    114006|       15|
|    151402|       46|
|    146879|       12|
|     81254|       10|
|     26368|        4|
|     90894|       28|
|     21340|        4|
|    144139|       42|
|    138040|       36|
|    105719|       19|
|    147165|       24|
|    116387|       24|
|     73064|       42|
|    110040|       45|
|    139131|       27|
|     80854|        3|
|    163519|        6|
|    102068|      224|
|    129988|       18|
|    130386|       47|
|    160008|       15|
|     93850|       65|
|    154624|       32|
|    106471|        2|
|    112714|       29|
|    104427|       32|
|    133709|       38|
|     55495

In [19]:
df_final = df_res.select(col("product_id"),stop_words_deletion(col("comment")).alias("comment") ) 
df_final.show(truncate=False)

+----------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|product_id|comment                                                                                                                                                                                                  

In [20]:
df_res.withColumn('word', F.explode(F.split(F.col('comment'), ' ')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=False)\
    .show(100)

+-------+-----+
|   word|count|
+-------+-----+
|استفاده| 1783|
|  کیفیت| 1594|
|   میشه| 1389|
|  خریدم| 1300|
|   گوشی| 1174|
|   خوبه| 1140|
|  عالیه| 1140|
|  واقعا| 1076|
|   اصلا|  981|
|  میکنم|  933|
|  نداره|  884|
|   قیمت|  884|
|  میکنه|  883|
|   خرید|  858|
|    کار|  837|
|   دیجی|  808|
|   راضی|  801|
|   شارژ|  792|
|پیشنهاد|  769|
|   کالا|  707|
|   باشه|  642|
|   دیگه|  635|
|  محصول|  629|
|   سلام|  615|
|  قیمتش|  565|
| دستگاه|  492|
|  گرفتم|  491|
|   دارم|  489|
|   ساعت|  464|
|   حتما|  464|
| دوستان|  455|
|    ماه|  454|
|    مدل|  453|
|    رنگ|  444|
|   الان|  440|
|  راضیم|  428|
|   ارزش|  413|
|    جنس|  408|
|   میده|  381|
|  نمیشه|  381|
|   بازی|  378|
|    کنه|  374|
|   هستش|  367|
|    سال|  366|
|   هستم|  352|
|    دست|  346|
|   مشکل|  330|
|  بخرید|  323|
|  خوبیه|  323|
|   دستم|  322|
|   راحت|  315|
|    نمی|  313|
|    بشه|  312|
|    بگم|  303|
|   توجه|  293|
| دوربین|  291|
|   شگفت|  290|
| العاده|  283|
|   صفحه|  280|
|   صدای

In [21]:
def tokenize(input_text):
  try:
    #lemmatizer = Lemmatizer()
    token_text = word_tokenize(input_text)
    #token_text_final = [lemmatizer.lemmatize(t) for t in token_text]
    token_text_final = list(set(token_text))
    return ','.join(token_text_final)
  except:
    return None

In [22]:
tokenize('سلام من خوبم تو خوبی؟')

'تو,؟,خوبی,خوبم,سلام,من'

In [23]:
products_sel = filter_products.select(col("product_id"),stop_words_deletion(col("comment")).alias("comment") ) 
products_sel.show(truncate=False)

+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [24]:
comment_tokenization = udf(lambda z: tokenize(z))
df_tokenize = products_sel.select(col("product_id"),comment_tokenization(col("comment")).alias("comment") ) 
df_tokenize.show(20)

+----------+--------------------+
|product_id|             comment|
+----------+--------------------+
|    517207|ماشین,حرکت,بگم,نخ...|
|    597475|ساخته,ساده,تشخیص,...|
|    503608|        نداشت,تاثیری|
|    701679|نقاش,رنگی,دیدم,کا...|
|    133901|دی,حسرت,قطعا,هاست...|
|    517207|ماشین,پراید,ایمن,...|
|    443966|خورشد,طلایی,کنه,خ...|
|    226249|ترجیحا,خدا,اصلی,ب...|
|    341816|قابلیت,کنسول,برعک...|
|    313420|    عرضه,نمی,بهقیمتش|
|    253710|نیس,ساخته,راحت,زم...|
|    203451|هوشمند,سرگرمی,زما...|
|    334514|امریکاییه,هرچندچی...|
|    700304|   انتظار,کلام,داشتم|
|    137928|تقلب,کشیدم,ساخته,...|
|    388255|میشه,برسه,پاره,ذر...|
|    189875|بیسیم,راحت,عرض,کر...|
|    700304|راضی,بودنش,واقعا,...|
|    135021|ماشالله,رفتن,سریع...|
|    619216|      لطفا,کامنت,پاک|
+----------+--------------------+
only showing top 20 rows



In [25]:
from pyspark.sql.functions import split, col
df2 = df_tokenize.select(split(col("comment"),",").alias("commentArray")) \
    .drop("comment")
df2.printSchema()
df2.show()

root
 |-- commentArray: array (nullable = true)
 |    |-- element: string (containsNull = true)

+--------------------+
|        commentArray|
+--------------------+
|[ماشین, حرکت, بگم...|
|[ساخته, ساده, تشخ...|
|     [نداشت, تاثیری]|
|[نقاش, رنگی, دیدم...|
|[دی, حسرت, قطعا, ...|
|[ماشین, پراید, ای...|
|[خورشد, طلایی, کن...|
|[ترجیحا, خدا, اصل...|
|[قابلیت, کنسول, ب...|
|[عرضه, نمی, بهقیمتش]|
|[نیس, ساخته, راحت...|
|[هوشمند, سرگرمی, ...|
|[امریکاییه, هرچند...|
|[انتظار, کلام, دا...|
|[تقلب, کشیدم, ساخ...|
|[میشه, برسه, پاره...|
|[بیسیم, راحت, عرض...|
|[راضی, بودنش, واق...|
|[ماشالله, رفتن, س...|
|  [لطفا, کامنت, پاک]|
+--------------------+
only showing top 20 rows



In [26]:
df_tokenize.printSchema

<bound method DataFrame.printSchema of DataFrame[product_id: string, comment: string]>

In [27]:

from pyspark.ml.fpm import FPGrowth

df_f = spark.createDataFrame([
    (0, [1, 2, 5]),
    (1, [1, 2, 3, 5]),
    (2, [1, 2])
], ["id", "items"])

fpGrowth = FPGrowth(itemsCol="commentArray", minSupport=0.1, minConfidence=0.1)
model = fpGrowth.fit(df2)

# Display frequent itemsets.
model.freqItemsets.show()

# Display generated association rules.
model.associationRules.show()

# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(df2).show()


/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+--------------------+----+
|               items|freq|
+--------------------+----+
|               [بشه]|   6|
|      [بشه, استفاده]|   5|
|              [مثلا]|   5|
|               [بگم]|   8|
|        [بگم, کیفیت]|   6|
|[بگم, کیفیت, استف...|   5|
|      [بگم, استفاده]|   6|
|              [دیجی]|   6|
|              [دیگه]|   6|
|        [دیگه, میشه]|   5|
|         [دیگه, کار]|   5|
|     [دیگه, استفاده]|   5|
|              [قیمت]|   7|
|              [میده]|   5|
|              [ساعت]|   6|
|              [سلام]|  10|
|     [سلام, استفاده]|   5|
|               [کنه]|   9|
|              [حتما]|   7|
|     [حتما, استفاده]|   6|
+--------------------+----+
only showing top 20 rows

+----------------+----------+------------------+------------------+-------+
|      antecedent|consequent|        confidence|              lift|support|
+----------------+----------+------------------+------------------+-------+
|          [سلام]| [استفاده]|               0.5|2.0833333333333335|    0.1

### Q4

In [ ]:

pdf = pd.read_csv("https://raw.githubusercontent.com/SajjadMb/PySpark-walkthrough/main/data/digikala_comments.csv")
selected = pdf.comment
df = pd.DataFrame(selected)

df

In [ ]:
from pyspark.sql.types import *
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)


# Auxiliar functions
def equivalent_type(f):
    if f == 'datetime64[ns]': return TimestampType()
    elif f == 'int64': return LongType()
    elif f == 'int32': return IntegerType()
    elif f == 'float64': return FloatType()
    else: return StringType()

def define_structure(string, format_type):
    try: typo = equivalent_type(format_type)
    except: typo = StringType()
    return StructField(string, typo)

# Given pandas dataframe, it will return a spark's dataframe.
def pandas_to_spark(pandas_df):
    columns = list(pandas_df.columns)
    types = list(pandas_df.dtypes)
    struct_list = []
    for column, typo in zip(columns, types): 
      struct_list.append(define_structure(column, typo))
    p_schema = StructType(struct_list)
    return sqlContext.createDataFrame(pandas_df, p_schema)

In [ ]:
res=pandas_to_spark(pdf)